In [ ]:
####################################################################################################
## This file is run once to create the 2021_04_01_week_associations_times.csv file from raw data
## This is the data preprocessing steps for one week of raw data
####################################################################################################

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandasql import sqldf
from sqlalchemy import create_engine

from datetime import datetime
from datetime import timedelta
import os, sys

import gzip

In [57]:
##################
# import raw data
##################
try:
    df = pd.read_csv('2021_04_01_week_associations.csv.gz', compression='gzip', warn_bad_lines=True)
except Exception as e:
    print(e)
    
df

,start_time,installation_id,station_id
0,2021-04-01 00:00:00.088007,412914,1025503
1,2021-04-01 00:00:00.107122,413884,985795
2,2021-04-01 00:00:00.199838,418743,876082
3,2021-04-01 00:00:00.244472,419447,883379
4,2021-04-01 00:00:00.352282,413584,855284
...,...,...,...
12978005,2021-04-07 23:59:59.800863,416813,1042530
12978006,2021-04-07 23:59:59.871990,419362,961954
12978007,2021-04-07 23:59:59.885904,415686,1042463
12978008,2021-04-07 23:59:59.912764,415020,870582


In [58]:
# get time zero
first_time = datetime.strptime(df.iloc[0]['start_time'], '%Y-%m-%d %H:%M:%S.%f')
first_time

datetime.datetime(2021, 4, 1, 0, 0, 0, 88007)

In [61]:
##################################################
# function for integer time since 0 on current day
# input: t, string of start_time of current row
# output: returns difference in seconds from start time (seconds since start)
##################################################
def time_since_start(t):
    try:
        current_time = datetime.strptime(t, '%Y-%m-%d %H:%M:%S.%f')
    except:
        t = t + ".0"
        current_time = datetime.strptime(t, '%Y-%m-%d %H:%M:%S.%f')
        
    difference = (current_time - first_time).total_seconds()
    return difference

In [62]:
# get dates as integers of seconds since time 0 on given day
df['time_stamp'] = df.apply(lambda row : time_since_start(row['start_time']), axis = 1)

In [63]:
# using encoding for time
df.drop(columns=['start_time'])
df

,start_time,installation_id,station_id,time_stamp
0,2021-04-01 00:00:00.088007,412914,1025503,0.000000
1,2021-04-01 00:00:00.107122,413884,985795,0.019115
2,2021-04-01 00:00:00.199838,418743,876082,0.111831
3,2021-04-01 00:00:00.244472,419447,883379,0.156465
4,2021-04-01 00:00:00.352282,413584,855284,0.264275
...,...,...,...,...
12978005,2021-04-07 23:59:59.800863,416813,1042530,604799.712856
12978006,2021-04-07 23:59:59.871990,419362,961954,604799.783983
12978007,2021-04-07 23:59:59.885904,415686,1042463,604799.797897
12978008,2021-04-07 23:59:59.912764,415020,870582,604799.824757


In [64]:
######################
# create db connection
######################
connection_to_db = create_engine('sqlite:///week_associations.db')

In [65]:
# table from raw data
df.to_sql('week_associations', con=connection_to_db, if_exists='replace')

In [66]:
# run first time
%load_ext sql
%sql sqlite:///week_associations.db

In [2]:
# otherwise run this
%reload_ext sql
%sql sqlite:///week_associations.db

In [ ]:
# sanity check

In [69]:
%%sql

SELECT * FROM week_associations LIMIT 5; 

 * sqlite:///week_associations.db
Done.


index,start_time,installation_id,station_id,time_stamp
0,2021-04-01 00:00:00.088007,412914,1025503,0.0
1,2021-04-01 00:00:00.107122,413884,985795,0.019115
2,2021-04-01 00:00:00.199838,418743,876082,0.111831
3,2021-04-01 00:00:00.244472,419447,883379,0.156465
4,2021-04-01 00:00:00.352282,413584,855284,0.264275


In [ ]:
# builds associations_cross table
# format:
# station_id, ap_1, ap_2, time_difference

In [3]:
%%sql

create table associations_cross as select
a.station_id, 
a.installation_id as ap_1,
b.installation_id as ap_2,
abs(b.time_stamp - a.time_stamp) as time_difference
from week_associations a join week_associations b on a.station_id = b.station_id
where a.installation_id != b.installation_id and abs(b.time_stamp - a.time_stamp) < 7200;

 * sqlite:///week_associations.db
Done.


[]

In [4]:
# sanity check

In [5]:
%%sql

SELECT * FROM associations_cross LIMIT 5; 

 * sqlite:///week_associations.db
Done.


station_id,ap_1,ap_2,time_difference
1025503,412914,412996,29.842606
1025503,412914,413581,76.468231
1025503,412914,413581,351.199376
1025503,412914,413581,597.040277
1025503,412914,413581,2264.635228


In [6]:
# join instances where ap_1 == ap_1 AND ap_2 == ap_2
# add col for count
# add col for sum of time_stamps
# add col for minimum time

In [7]:
%%sql
create table associations_results as select
ap_1,
ap_2,
sum(time_difference) as total_time,
count(*) as frequency,
min(time_difference) as min_time
from associations_cross
group by
ap_1,
ap_2;

 * sqlite:///week_associations.db
Done.


[]

In [8]:
# sanity check

In [9]:
%%sql

SELECT * FROM associations_results LIMIT 5; 

 * sqlite:///week_associations.db
Done.


ap_1,ap_2,total_time,frequency,min_time
1106,412028,1150.4628760000342,1,1150.4628760000342
1106,412046,8666.911970000001,2,4332.140609000002
1106,412126,15732.078110000002,4,3565.275391000003
1106,412130,22974.635945999995,6,3524.078822999996
1106,412135,1147.9935530000366,1,1147.9935530000366


In [10]:
# export csv or zip file if too big

In [11]:
result = %sql SELECT * FROM associations_results
result_df = result.DataFrame()

 * sqlite:///week_associations.db
Done.


In [12]:
cwd = os.getcwd()
path = cwd + "/2021_04_01_week_associations_times.csv"
result_df.to_csv(path, index=False)